<a href="https://colab.research.google.com/github/NMJ0/Detailed-Router/blob/main/detailed_router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required Installations and imports**

In [ ]:
!pip install --break-system-packages https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/project/def/c17.def
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/lef/sky130.lef

In [31]:
import requests
import LEFDEFParser
from LEFDEFParser import Rect


**Parser for global routing solutions**.\
Returns a dictionary with net names as keys and their global routing solutions as values

In [54]:
url = "https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/gr/c17.guide"
response = requests.get(url)
lines = response.text.splitlines()
global_route={}
n=len(lines)
m=0
while True:
  if m>=n-1:
    break
  name=lines[m]
  global_route[name]=[]
  m+=1
  while True:
    if lines[m]==')':
      m+=1
      break
    if lines[m]=='(':
      m+=1
    x1,y1,x2,y2,layer=map(str,lines[m].split())
    global_route[name].append({layer:[(int(x1),int(y1)),(int(x2),int(y2))]})
    m+=1
print(global_route)
#global_route

{'N1': [{'li1': [(13800, 20700), (20700, 27600)]}, {'met1': [(13800, 20700), (20700, 27600)]}], 'N1_d': [{'met2': [(0, 20700), (6900, 34500)]}, {'met1': [(0, 27600), (6900, 34500)]}, {'met1': [(0, 27600), (13800, 34500)]}, {'li1': [(6900, 27600), (13800, 34500)]}, {'met2': [(0, 20700), (6900, 41420)]}], 'N2': [{'li1': [(13800, 20700), (20700, 27600)]}, {'met1': [(13800, 20700), (20700, 27600)]}], 'N22': [{'li1': [(6900, 6900), (13800, 13800)]}, {'met1': [(6900, 6900), (13800, 13800)]}, {'met2': [(6900, 0), (13800, 13800)]}], 'N22_d': [{'li1': [(13800, 20700), (20700, 27600)]}, {'met1': [(13800, 20700), (20700, 27600)]}, {'met2': [(13800, 6900), (20700, 27600)]}, {'met1': [(13800, 6900), (20700, 13800)]}, {'li1': [(13800, 6900), (20700, 13800)]}], 'N23': [{'li1': [(20700, 6900), (30700, 13800)]}, {'met1': [(20700, 6900), (30700, 13800)]}, {'met2': [(20700, 0), (30700, 13800)]}], 'N23_d': [{'li1': [(13800, 13800), (20700, 20700)]}, {'met1': [(13800, 13800), (20700, 20700)]}, {'met2': [(1

**Skip Cells and Skip Nets**

In [37]:
skipCells = {"sky130_fd_sc_hd__decap_3", "sky130_fd_sc_hd__decap_4", "sky130_fd_sc_hd__decap_6", "sky130_fd_sc_hd__decap_8",\
            "sky130_fd_sc_hd__decap_12", "sky130_fd_sc_hd__fill_1", "sky130_fd_sc_hd__fill_2", "sky130_fd_sc_hd__fill_4",
            "sky130_fd_sc_hd__fill_8", "sky130_fd_sc_hd__lpflow_decapkapwr_3", "sky130_fd_sc_hd__lpflow_decapkapwr_4",\
            "sky130_fd_sc_hd__lpflow_decapkapwr_6", "sky130_fd_sc_hd__lpflow_decapkapwr_8", "sky130_fd_sc_hd__lpflow_decapkapwr_12", \
            "sky130_fd_sc_hd__lpflow_lsbuf_lh_hl_isowell_tap_1", "sky130_fd_sc_hd__lpflow_lsbuf_lh_hl_isowell_tap_2", \
            "sky130_fd_sc_hd__lpflow_lsbuf_lh_hl_isowell_tap_4", "sky130_fd_sc_hd__lpflow_lsbuf_lh_isowell_tap_1", \
            "sky130_fd_sc_hd__lpflow_lsbuf_lh_isowell_tap_2", "sky130_fd_sc_hd__lpflow_lsbuf_lh_isowell_tap_4", "sky130_fd_sc_hd__tap_1", \
            "sky130_fd_sc_hd__tap_2", "sky130_fd_sc_hd__tapvgnd2_1", "sky130_fd_sc_hd__tapvgnd_1", \
            "sky130_fd_sc_hd__tapvpwrvgnd_1", "sky130_ef_sc_hd__decap_12"}

layerColors = { 'li1': 'red', 'met1': 'blue', 'met2': 'green', 'met3': 'orange', 'met4': 'magenta', 'met5': 'cyan' }
layerOrient = { 'li1': 'VERTICAL', 'met1': 'HORIZONTAL', 'met2': 'VERTICAL', 'met3': 'HORIZONTAL', 'met4': 'VERTICAL', 'met5': 'HORIZONTAL' }

# skip power/ground/clock nets
skipNets = {'clk', 'VPWR', 'VGND'}

adjLayer = {
  'li1':  ['met1'],
  'met1': ['li1',  'met2'],
  'met2': ['met1', 'met3'],
  'met3': ['met2', 'met4'],
  'met4': ['met3', 'met5'],
  'met5': ['met4']
}

**DEF and LEF reader. Reading cells and nets.**

In [70]:
l = LEFDEFParser.LEFReader()
leffile="sky130.lef"
l.readLEF(leffile)
d = LEFDEFParser.DEFReader()
deffile="c17.def"
d.readDEF(deffile)

macros={}
for i in l.macros():
  macros[i.name()]=i

cells={}
for i in d.components():
  cells[i]=i.macro()

nets={}
for i in d.nets():
  nets[i.name()]=i.pins()
print(macros)
print(cells)
print(nets)
#cells
#nets


{'sky130_fd_sc_hd__a2bb2o_1': <LEFDEFParser.Macro object at 0x7d892e5b39f0>, 'sky130_fd_sc_hd__a2bb2o_2': <LEFDEFParser.Macro object at 0x7d892e5b2c70>, 'sky130_fd_sc_hd__a2bb2o_4': <LEFDEFParser.Macro object at 0x7d892e5b1330>, 'sky130_fd_sc_hd__a2bb2oi_1': <LEFDEFParser.Macro object at 0x7d892e5b0870>, 'sky130_fd_sc_hd__a2bb2oi_2': <LEFDEFParser.Macro object at 0x7d892e5b3030>, 'sky130_fd_sc_hd__a2bb2oi_4': <LEFDEFParser.Macro object at 0x7d892e5b1670>, 'sky130_fd_sc_hd__a21bo_1': <LEFDEFParser.Macro object at 0x7d892e5b37b0>, 'sky130_fd_sc_hd__a21bo_2': <LEFDEFParser.Macro object at 0x7d892e5b3a30>, 'sky130_fd_sc_hd__a21bo_4': <LEFDEFParser.Macro object at 0x7d892e5b18f0>, 'sky130_fd_sc_hd__a21boi_0': <LEFDEFParser.Macro object at 0x7d892e5b3930>, 'sky130_fd_sc_hd__a21boi_1': <LEFDEFParser.Macro object at 0x7d892e5b1bf0>, 'sky130_fd_sc_hd__a21boi_2': <LEFDEFParser.Macro object at 0x7d892e5b3130>, 'sky130_fd_sc_hd__a21boi_4': <LEFDEFParser.Macro object at 0x7d892e5b2070>, 'sky130_fd_

**Transforming pin locations  and obstructions acccording to cell orientation and storing it in a dictionary**

In [58]:
class Inst:
  def __init__(self, comp, macro):
    self._comp = comp
    self._macro = macro
    origin = comp.location()
    self._bbox = Rect(origin.x, origin.y, origin.x + macro.xdim(), origin.y + macro.ydim())
    self._pins = dict()
    self._obsts = dict()
    for p in macro.pins():
      shapes = dict()
      for port in p.ports():
        for layer, rects in port.items():
          if layer not in layerColors: continue
          if layer not in shapes: shapes[layer] = list()
          for v in rects:
            r = Rect(v.ll.x, v.ll.y, v.ur.x, v.ur.y)
            r.transform(comp.orient(), origin, macro.xdim(), macro.ydim())
            shapes[layer].append(r)
      self._pins[p.name()] = shapes

    for layer, rects in macro.obstructions().items():
      if layer not in layerColors: continue
      if layer not in self._obsts: self._obsts[layer] = list()
      for v in rects:
        r = Rect(v.ll.x, v.ll.y, v.ur.x, v.ur.y)
        r.transform(comp.orient(), origin, macro.xdim(), macro.ydim())
        self._obsts[layer].append(r)
cell_pin={}
obstructions={'li1':[],'met1':[],'met2':[],'met3':[],'met4':[],'met5':[]}
for l in cells.keys():
  inst=Inst(l,macros[cells[l]])
  for j in inst._obsts.keys():
    obstructions[j].extend(inst._obsts[j])
  for i in inst._pins.keys():
    for k in inst._pins[i].keys():
      obstructions[k].extend(inst._pins[i][k])
    if len(inst._pins[i])==0:
      continue
    cell_pin[(inst._comp.name(),i)]=inst._pins[i]
print(cell_pin)
print(obstructions)


{('PHY_EDGE_ROW_0_Left_7', 'VGND'): {'met1': [[(5520,10640),(6900,11120)]]}, ('PHY_EDGE_ROW_0_Left_7', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VGND'): {'met1': [[(23460,10640),(24840,11120)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VPWR'): {'met1': [[(5520,18800),(6900,19280)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VPWR'): {'met1': [[(23460,18800),(24840,19280)]]}, ('PHY_EDGE_ROW_3_Left_10', 'VGND'): {'met1': [[(5520,21520),(6900,220

**Adding boundary pins**

In [64]:
b_pins={}
for i in d.pins():
  b_pins['PIN',i.name()]=i.ports()[0]

for i in b_pins:
  cell_pin[i]=b_pins[i]
  for j in b_pins[i].keys():
    obstructions[j].extend(b_pins[i][j])
print(cell_pin)
print(obstructions)

{('PHY_EDGE_ROW_0_Left_7', 'VGND'): {'met1': [[(5520,10640),(6900,11120)]]}, ('PHY_EDGE_ROW_0_Left_7', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VGND'): {'met1': [[(23460,10640),(24840,11120)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VPWR'): {'met1': [[(5520,18800),(6900,19280)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VPWR'): {'met1': [[(23460,18800),(24840,19280)]]}, ('PHY_EDGE_ROW_3_Left_10', 'VGND'): {'met1': [[(5520,21520),(6900,220

**Extracting track information, layer width and spacing.**

In [69]:
layerWidth = {}
layerSpacing = {}
for layer in l.layers():
    layerWidth[layer.name()] = layer.width()
    layerSpacing[layer.name()] = layer.pitch() - layer.width()
tracks=d.tracks()
tracks_ = {
    'li1':  [tracks['li1'][0].x,  tracks['li1'][0].num,  tracks['li1'][0].step],
    'met1': [tracks['met1'][1].x, tracks['met1'][1].num, tracks['met1'][1].step],
    'met2': [tracks['met2'][0].x, tracks['met2'][0].num, tracks['met2'][0].step],
    'met3': [tracks['met3'][1].x, tracks['met3'][1].num, tracks['met3'][1].step],
    'met4': [tracks['met4'][0].x, tracks['met4'][0].num, tracks['met4'][0].step],
    'met5': [tracks['met5'][1].x, tracks['met5'][1].num, tracks['met5'][1].step],
}
print(tracks_)
print(layerWidth)
print(layerSpacing)

{'li1': [230, 67, 460], 'met1': [170, 122, 340], 'met2': [230, 67, 460], 'met3': [340, 61, 680], 'met4': [460, 33, 920], 'met5': [1700, 12, 3400]}
{'li1': 170, 'met1': 140, 'met2': 140, 'met3': 300, 'met4': 300, 'met5': 1600}
{'li1': 170, 'met1': 200, 'met2': 320, 'met3': 380, 'met4': 620, 'met5': 1800}


**Converting Netlist into rectangular regions that are to be connected .**